[click mi](https://www.rabbitmq.com/getstarted.html)

In [ ]:
#worker
import pika

def callback(ch, method, propertis, body):
    print("[x] Recived %r" % body)

def main():
    credentials = pika.PlainCredentials('rabbitmq', 'rabbitmq')# login password
    parameters = pika.ConnectionParameters('rabbitmq', 5672, '/', credentials)
    connection = pika.BlockingConnection(parameters)
    
    channel = connection.channel()
    channel.queue_declare(queue='queuel')
    
    channel.basic_consume(queue='queuel', on_message_callback=callback, auto_ack=True)
    #auto_ack - автоподьверждение выполненного таска
    
    print("[*] Waiting for message. To exit press CTRL+C")
    channel.start_consuming()

main()

***

In [ ]:
#worker
import pika
import time

credentials = pika.PlainCredentials('rabbitmq', 'rabbitmq')# login password
parameters = pika.ConnectionParameters('rabbitmq', 5672, '/', credentials)
connection = pika.BlockingConnection(parameters)
channel = connection.channel()

channel.queue_declare(queue='task_queue', durable=True)#сообщения в очереди долговечны
print(' [*] Waiting for messages. To exit press CTRL+C')


def callback(ch, method, properties, body):
    print(" [x] Received %r" % body)
    time.sleep(body.count(b'.'))
    print(" [x] Done")
    ch.basic_ack(delivery_tag=method.delivery_tag)#подтверждение того что сообщение было получено и обработано

while True:
    channel.basic_qos(prefetch_count=1)# не принимать более одного сообщения
    channel.basic_consume(queue='task_queue', on_message_callback=callback)

    channel.start_consuming()

 [*] Waiting for messages. To exit press CTRL+C
 [x] Received b'Hello..0'
 [x] Done
 [x] Received b'Hello..1'
 [x] Done
 [x] Received b'Hello..3'
 [x] Done
 [x] Received b'Hello..5'
 [x] Done
 [x] Received b'Hello..7'
 [x] Done
 [x] Received b'Hello..9'
 [x] Done
 [x] Received b'Hello..11'
 [x] Done
 [x] Received b'Hello..13'
 [x] Done
 [x] Received b'Hello..14'
 [x] Done
 [x] Received b'Hello..15'
 [x] Done
 [x] Received b'Hello..16'
 [x] Done
 [x] Received b'Hello..17'
 [x] Done
 [x] Received b'Hello..18'
 [x] Done
 [x] Received b'Hello..19'
 [x] Done
 [x] Received b'Hello..20'
 [x] Done
 [x] Received b'Hello..21'
 [x] Done
 [x] Received b'Hello..22'
 [x] Done
 [x] Received b'Hello..23'
 [x] Done
 [x] Received b'Hello..24'
 [x] Done
 [x] Received b'Hello..25'
 [x] Done
 [x] Received b'Hello..26'
 [x] Done
 [x] Received b'Hello..27'
 [x] Done
 [x] Received b'Hello..28'
 [x] Done
 [x] Received b'Hello..29'
 [x] Done
 [x] Received b'Hello..31'
 [x] Done
 [x] Received b'Hello..33'
 [x] D

***

In [ ]:
import pika

credentials = pika.PlainCredentials('rabbitmq', 'rabbitmq')# login password
parameters = pika.ConnectionParameters('rabbitmq', 5672, '/', credentials)
connection = pika.BlockingConnection(parameters)
channel = connection.channel()

channel.exchange_declare(exchange='logs', exchange_type='fanout')

result = channel.queue_declare(queue='', exclusive=True)  #гинерит имя очереди
# exclusive - удалить очередь ро завершению сеанса
queue_name = result.method.queue                         # получаем имя очереди

channel.queue_bind(exchange='logs', queue=queue_name)

print(' [*] Waiting for logs. To exit press CTRL+C')

def callback(ch, method, properties, body):
    print(" [x] %r" % body)

while True:
    channel.basic_consume(queue=queue_name, on_message_callback=callback, auto_ack=True)
    channel.start_consuming()

 [*] Waiting for logs. To exit press CTRL+C
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info: Hello World!'
 [x] b'info

***

In [ ]:
import pika
import sys

credentials = pika.PlainCredentials('rabbitmq', 'rabbitmq')# login password
parameters = pika.ConnectionParameters('rabbitmq', 5672, '/', credentials)
connection = pika.BlockingConnection(parameters)
channel = connection.channel()

channel.exchange_declare(exchange='direct_logs', exchange_type='direct')

result = channel.queue_declare(queue='', exclusive=True)
queue_name = result.method.queue

severities = 'info', 'warning'


for severity in severities:
    channel.queue_bind(exchange='direct_logs', queue=queue_name, routing_key=severity)

print(' [*] Waiting for logs. To exit press CTRL+C')


def callback(ch, method, properties, body):
    print(" [x] %10r:%r" % (method.routing_key, body))

while True:
    channel.basic_consume(queue=queue_name, on_message_callback=callback, auto_ack=True)
    channel.start_consuming()

 [*] Waiting for logs. To exit press CTRL+C
 [x]     'info':b'Hello World! 0'
 [x]  'warning':b'Hello World! 1'
 [x]     'info':b'Hello World! 3'
 [x]  'warning':b'Hello World! 4'
 [x]     'info':b'Hello World! 6'
 [x]  'warning':b'Hello World! 7'
 [x]     'info':b'Hello World! 9'
 [x]  'warning':b'Hello World! 10'
 [x]     'info':b'Hello World! 12'
 [x]  'warning':b'Hello World! 13'
 [x]     'info':b'Hello World! 15'
 [x]  'warning':b'Hello World! 16'
 [x]     'info':b'Hello World! 18'
 [x]  'warning':b'Hello World! 19'
 [x]     'info':b'Hello World! 21'
 [x]  'warning':b'Hello World! 22'
 [x]     'info':b'Hello World! 24'
 [x]  'warning':b'Hello World! 25'
 [x]     'info':b'Hello World! 27'
 [x]  'warning':b'Hello World! 28'
 [x]     'info':b'Hello World! 30'
 [x]  'warning':b'Hello World! 31'
 [x]     'info':b'Hello World! 33'
 [x]  'warning':b'Hello World! 34'
 [x]     'info':b'Hello World! 36'
 [x]  'warning':b'Hello World! 37'
 [x]     'info':b'Hello World! 39'
 [x]  'warning':b'

***

In [ ]:
import pika
import sys

credentials = pika.PlainCredentials('rabbitmq', 'rabbitmq')# login password
parameters = pika.ConnectionParameters('rabbitmq', 5672, '/', credentials)
connection = pika.BlockingConnection(parameters)
channel = connection.channel()

channel.exchange_declare(exchange='topic_logs', exchange_type='topic')

result = channel.queue_declare('', exclusive=True)
queue_name = result.method.queue

binding_keys = ['anonymous.*']


for binding_key in binding_keys:
    channel.queue_bind(exchange='topic_logs', queue=queue_name, routing_key=binding_key)

print(' [*] Waiting for logs. To exit press CTRL+C')


def callback(ch, method, properties, body):
    print(" [x] %r:%r" % (method.routing_key, body))

while True:
    channel.basic_consume(queue=queue_name, on_message_callback=callback, auto_ack=True)
    channel.start_consuming()

 [*] Waiting for logs. To exit press CTRL+C
 [x] 'anonymous.info':b'Hello World!0'
 [x] 'anonymous.info':b'Hello World!1'
 [x] 'anonymous.info':b'Hello World!2'
 [x] 'anonymous.info':b'Hello World!3'
 [x] 'anonymous.info':b'Hello World!4'
 [x] 'anonymous.info':b'Hello World!5'
 [x] 'anonymous.info':b'Hello World!6'
 [x] 'anonymous.info':b'Hello World!7'
 [x] 'anonymous.info':b'Hello World!8'
 [x] 'anonymous.info':b'Hello World!9'
 [x] 'anonymous.info':b'Hello World!10'
 [x] 'anonymous.info':b'Hello World!11'
 [x] 'anonymous.info':b'Hello World!12'
 [x] 'anonymous.info':b'Hello World!13'
 [x] 'anonymous.info':b'Hello World!14'
 [x] 'anonymous.info':b'Hello World!15'
 [x] 'anonymous.info':b'Hello World!16'
 [x] 'anonymous.info':b'Hello World!17'
 [x] 'anonymous.info':b'Hello World!18'
 [x] 'anonymous.info':b'Hello World!19'
 [x] 'anonymous.info':b'Hello World!20'
 [x] 'anonymous.info':b'Hello World!21'
 [x] 'anonymous.info':b'Hello World!22'
 [x] 'anonymous.info':b'Hello World!23'
 [x] '

***